In [24]:
import numpy as np
from scipy.optimize import minimize
import scipy.stats as stats
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
from mpl_toolkits.basemap import Basemap

In [25]:
%run data_processing.ipynb

In [26]:
def process_data(data):
    d = data[['1st Max Value', 'GC', 'uniqueID']].values
    return d

In [27]:
# code for ridge regression
def ridge_regression(X, y, l):
    # construct modified X and y
    X_mod = np.concatenate((X, np.sqrt(l)*np.identity(X.shape[1])),axis=0) 
    y_mod = np.concatenate((y, np.zeros(X.shape[1])), axis=0)
    
    # take QR factorization of modified X
    Q, R = np.linalg.qr(X_mod, mode='reduced')
    
    # compute w_hat
    w_hat = np.mat(np.linalg.inv(R)) * np.mat(np.transpose(Q)) * np.transpose(np.mat(y_mod))
    
    return w_hat

In [28]:
# get RMSE
def compute_RMSE(X, y, w_hat):
    RMSE = np.sum(np.square(np.transpose(y)-np.transpose(w_hat)*np.transpose(X)))
    RMSE = np.sqrt(RMSE / y.shape[0])
    return RMSE

In [29]:
# read data
df = pd.read_csv('daily_44201_2013.csv')
gc1 = np.loadtxt('gc4x5jan.csv', skiprows=0, delimiter=',')
gc2 = np.loadtxt('gc4x5feb.csv', skiprows=0, delimiter=',')
gc3 = np.loadtxt('gc4x5mar.csv', skiprows=0, delimiter=',')
gc4 = np.loadtxt('gc4x5apr.csv', skiprows=0, delimiter=',')
gc5 = np.loadtxt('gc4x5may.csv', skiprows=0, delimiter=',')
gc6 = np.loadtxt('gc4x5jun.csv', skiprows=0, delimiter=',')
gc7 = np.loadtxt('gc4x5jul.csv', skiprows=0, delimiter=',')
gc8 = np.loadtxt('gc4x5aug.csv', skiprows=0, delimiter=',')
gc9 = np.loadtxt('gc4x5sep.csv', skiprows=0, delimiter=',')
gcsites = pd.read_csv('AQS_sites.csv', skiprows=0, delimiter=',')

# get only aug and sep data
ind = (df['Date Local'] >= '2013-01-01') & (df['Date Local'] <= '2013-09-30')
df = df[ind]

# add geos-chem data to df
df = add_gc(df, gcsites, gc1, gc2, gc3, gc4, gc5, gc6, gc7, gc8, gc9)

# remove zeros
ind = df['GC'] > 0
df = df[ind]

In [30]:
# separate into training and test set
d = process_data(df)

# get list of sites
sites = np.unique(np.array(d[:,2]))

RMSE = np.zeros(len(sites))
i = 0

# do ridge regression one site at a time
for s in sites:
    ind = np.where(np.squeeze(np.array(d[:,2])) == s)
    dd = d[ind]
    
    y_training, y_test, X_training, X_test = separate_data(dd)
    
    # do ridge regression
    w_hat = ridge_regression(X_training,y_training,0.5)
    
    # compute RMSE
    RMSE[i] = compute_RMSE(X_test,y_test,w_hat)
    i += 1


In [31]:
ind = np.where(np.isfinite(RMSE))
print(np.mean(RMSE[ind]))
print(np.max(RMSE[ind]))
print(np.min(RMSE[ind]))

8.81402914914
17.8763094118
3.51019981838


In [23]:
# plot correlation between GEOS-Chem and AQS data
x = np.copy(d[0:50000,0])
y = np.copy(d[0:50000,1])

font = {'family' : 'normal',
        'weight' : 'bold', 
        'size' : 16}
matplotlib.rc('font', **font)
axes = plt.gca()
axes.set_xlim([0,120])
axes.set_ylim([0,120])
plt.scatter(x*1000.0, y, s=1, c='black', marker=',')
plt.title('AQS vs. GEOS-Chem MDA8 ozone for Aug. - Sep. 2013')
plt.xlabel('AQS MDA8 ozone (ppb)')
plt.ylabel('GEOS-Chem MDA8 ozone (ppb)')
plt.savefig('AQSvGC.png')
plt.close()

/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):
/opt/local/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/matplotlib/font_manager.py:1282: UserWarning: findfont: Font family [u'normal'] not found. Falling back to Bitstream Vera Sans
  (prop.get_family(), self.defaultFamily[fontext]))


In [36]:
# plot GEOS-Chem RMSE without any sort of regression
err = df['1st Max Value']*1000.0 - df['GC']
sites, site_ind = np.unique(np.array(df['uniqueID']), return_index=True)
e_site = np.zeros(len(sites))
i = 0
n_tot = 0
# loop through all sites to get RMSE for each site
for s in sites:
    ind = np.where(np.squeeze(np.array(df['uniqueID'])) == s)
    if (len(ind[0]) > 1):
        ind = np.squeeze(ind)
        e_site[i] = np.sum((err[ind])**2) #/ len(ind))
        n_tot += len(ind)
        
    i += 1
    
Lat = df['Latitude'][site_ind]
Lon = df['Longitude'][site_ind]


In [37]:
print(n_tot)
ind = np.where(np.isfinite(e_site))
print(np.sum(e_site[ind])/n_tot)
print(np.max(e_site[ind]))
a = e_site[ind]
print(np.min(a[np.where(a>0)]))

275868
86.6930170402
155879.850943
9.33756553287


In [278]:
fig = plt.figure()
ax = fig.add_axes([0.05,0.05,0.9,0.9])
m = Basemap(width=12000000,height=9000000,projection='lcc',
            resolution='c',lat_1=45.,lat_2=55,lat_0=50,lon_0=-107.)
m.drawmapboundary(fill_color='0.3')
m.fillcontinents(color='coral',lake_color='aqua')
parallels = np.arange(0.,81,10.)
m.drawparallels(parallels,labels=[False,True,True,False])
meridians = np.arange(10.,351.,20.)
m.drawmeridians(meridians,labels=[True,False,False,True])
x,y=m(Lon.values, Lat.values)
m.pcolor(x,y, e_site, marker='D',color='m')
plt.show()

ValueError: need more than 1 value to unpack

In [297]:
np.savetxt('baseline_RMSE.csv', RMSE, delimiter=',')

In [298]:
np.savetxt('lats.csv', Lat.values, delimiter=',')
np.savetxt('lons.csv', Lon.values, delimiter=',')

In [54]:
np.max(d[:,1])

79.915381999999994